# # China Real Estate Demand Prediction — Pipeline Skeleton
#
# **Last generated:** 2025-10-06 07:47 UTC
#
# This notebook/script is a lean, safe baseline skeleton tailored to the Kaggle
# competition *"Real Estate Demand Prediction"*.
#
# ### What you get
# - Robust **IO setup** targeting Kaggle input paths
# - **Custom competition metric** (two-stage MAPE-based)
# - **Leakage-safe** rolling time-grouped cross-validation
# - Minimal **feature factory** with lags/rollings (extend here)
# - **Naïve** baseline (strong Stage-1 shield)
# - **LightGBM Tweedie** model scaffold + optional XGB/CatBoost hooks
# - **Blending + clipping** for metric safety
# - **Submission writer** preserving `test.csv` row order
#
# > Notes:
# > - Extend features in the marked sections. Keep temporal embargo ≥ max lag.
# > - If the official epsilon/edge-case handling differs, update the metric function accordingly.
#
# ---

In [3]:
import os, gc, sys, math, warnings, itertools, json
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

try:
    import lightgbm as lgb
except Exception as e:
    lgb = None

try:
    import xgboost as xgb
except Exception as e:
    xgb = None

try:
    from catboost import CatBoostRegressor, Pool as CatPool
except Exception as e:
    CatBoostRegressor, CatPool = None, None

In [4]:
class CFG:
    seed = 42
    n_folds = 5
    embargo_months = 3             # >= max lag window used in features
    use_target_lags = True         # if True, recursive inference to populate target lags
    use_tweedie = True             # main LightGBM objective
    lgb_params = dict(
        objective = "tweedie",
        tweedie_variance_power = 1.3,  # tune 1.3~1.7
        metric = "mae",
        num_leaves = 63,
        max_depth = 8,
        learning_rate = 0.05,
        feature_fraction = 0.6,
        bagging_fraction = 0.8,
        bagging_freq = 1,
        min_data_in_leaf = 128,
        lambda_l1 = 0.0,
        lambda_l2 = 5.0,
        n_estimators = 3000,
        verbose = -1,
        random_state = seed
    )

    data_dir = "/kaggle/input/china-real-estate-demand-prediction"
    out_dir  = "/kaggle/working"
    target_col = "amount_new_house_transactions"  # train target column name
    id_col     = "id"
    month_col  = "month"
    sector_col = "sector"
    sector_int_col = "sector_int"
    # files
    files = dict(
        new_house="train/new_house_transactions.csv",
        new_house_nb="train/new_house_transactions_nearby_sectors.csv",
        pre_owned="train/pre_owned_house_transactions.csv",
        pre_owned_nb="train/pre_owned_house_transactions_nearby_sectors.csv",
        land="train/land_transactions.csv",
        land_nb="train/land_transactions_nearby_sectors.csv",
        poi="train/sector_POI.csv",
        search="train/city_search_index.csv",
        city="train/city_indexes.csv",
        test="test.csv",
        sample="sample_submission.csv"
    )